In [81]:
import pandas as pd

In [82]:
df = pd.read_csv('AviationData.csv', header=0, encoding='cp1252',dtype=str) #'cp1252' encoding becuase of weird ascii values 

In [83]:
df.columns = df.columns.str.replace('.', ' ') # Change '.' in column names to ' '
df.columns = df.columns.str.title() # Change to title case
df.rename(columns={'Far Description': 'FAR Description', 'Event Id': 'Event ID'}, inplace = True) # Deal with the exceptions (ID, FAR)
pd.set_option('display.max_columns', 100) # Set max display width to 100
df.head() 
df

,Event ID,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,Injury Severity,Aircraft Damage,Aircraft Category,Registration Number,Make,Model,Amateur Built,Number Of Engines,Engine Type,FAR Description,Schedule,Purpose Of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase Of Flight,Report Status,Publication Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,NC6404,Stinson,108-3,No,1,Reciprocating,NaN,NaN,Personal,NaN,2,0,0,0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),Destroyed,NaN,N5069P,Piper,PA24-180,No,1,Reciprocating,NaN,NaN,Personal,NaN,4,0,0,0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),Destroyed,NaN,N5142R,Cessna,172M,No,1,Reciprocating,NaN,NaN,Personal,NaN,3,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,N1168J,Rockwell,112,No,1,Reciprocating,NaN,NaN,Personal,NaN,2,0,0,0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),Destroyed,NaN,N15NY,Cessna,501,No,NaN,NaN,NaN,NaN,Personal,NaN,1,2,NaN,0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,Minor,NaN,NaN,N1867H,PIPER,PA-28-151,No,NaN,NaN,091,NaN,Personal,NaN,0,1,0,0,NaN,NaN,NaN,29-12-2022
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N2895Z,BELLANCA,7ECA,No,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,Non-Fatal,Substantial,Airplane,N749PJ,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1,NaN,091,NaN,Personal,NaN,0,0,0,1,VMC,NaN,NaN,27-12-2022
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N210CU,CESSNA,210N,No,NaN,NaN,091,NaN,Personal,MC CESSNA 210N LLC,0,0,0,0,NaN,NaN,NaN,NaN


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Event ID                88889 non-null  object
 1   Investigation Type      88889 non-null  object
 2   Accident Number         88889 non-null  object
 3   Event Date              88889 non-null  object
 4   Location                88837 non-null  object
 5   Country                 88663 non-null  object
 6   Latitude                34382 non-null  object
 7   Longitude               34373 non-null  object
 8   Airport Code            50132 non-null  object
 9   Airport Name            52704 non-null  object
 10  Injury Severity         87889 non-null  object
 11  Aircraft Damage         85695 non-null  object
 12  Aircraft Category       32287 non-null  object
 13  Registration Number     87507 non-null  object
 14  Make                    88826 non-null  object
 15  Mo

In [85]:
df.isnull().sum().sort_values(ascending = False) # Ranked missing data

Schedule                  76307
Air Carrier               72241
FAR Description           56866
Aircraft Category         56602
Longitude                 54516
Latitude                  54507
Airport Code              38757
Airport Name              36185
Broad Phase Of Flight     27165
Publication Date          13771
Total Serious Injuries    12510
Total Minor Injuries      11933
Total Fatal Injuries      11401
Engine Type                7096
Report Status              6384
Purpose Of Flight          6192
Number Of Engines          6084
Total Uninjured            5912
Weather Condition          4492
Aircraft Damage            3194
Registration Number        1382
Injury Severity            1000
Country                     226
Amateur Built               102
Model                        92
Make                         63
Location                     52
Investigation Type            0
Event Date                    0
Accident Number               0
Event ID                      0
dtype: i

In [86]:
"""
Things we don't care about:

    Schedule
    FAR Description
    Aircraft Category:
        Helicopters
    Longitude
    Latitude
    Airport Code
    Airport Name
    Location
    Broad phase of flight
    Publication Date
    **Weather condition** maybe important 
    Air Carrier
    Registration number
    Country
    Location
    Investigation type
    **Event date** maybe important
    Accident number
    Event ID
    
Things we do care about:

    Total fatal injuries
    Total minor injuries
    Total serious injuries
    Aircraft Damage
    Engine Type
    Injury severity
    **Ameteur built** remove any data tied to this
    Model
    Make
    **Report status** some crashes were due to health issues
    
"""

"\nThings we don't care about:\n\n    Schedule\n    FAR Description\n    Aircraft Category:\n        Helicopters\n    Longitude\n    Latitude\n    Airport Code\n    Airport Name\n    Location\n    Broad phase of flight\n    Publication Date\n    **Report status** not sure what it means\n    Weather condition \n    Air Carrier\n    Registration number\n    Country\n    Location\n    Investigation type\n    **Event date** again, maybe important\n    Accident number\n    Event ID\n\n    * all starred columns are not in cols_to_drop below\n    \nThings we do care about:\n\n    Total fatal injuries\n    Total minor injuries\n    Total serious injuries\n    Aircraft Damage\n    Engine Type\n    Injury severity\n    **Ameteur built** remove any data tied to this\n    Model\n    Make\n    \n"

In [87]:
cols_to_drop = ['Schedule','FAR Description','Longitude','Latitude','Airport Code',
                'Airport Name','Location','Broad Phase Of Flight','Publication Date',
                'Weather Condition','Air Carrier','Registration Number','Country',
                'Location','Investigation Type','Accident Number','Event ID']

df = df.drop(cols_to_drop, axis = 1)
df

,Event Date,Injury Severity,Aircraft Damage,Aircraft Category,Make,Model,Amateur Built,Number Of Engines,Engine Type,Purpose Of Flight,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Report Status
0,1948-10-24,Fatal(2),Destroyed,NaN,Stinson,108-3,No,1,Reciprocating,Personal,2,0,0,0,Probable Cause
1,1962-07-19,Fatal(4),Destroyed,NaN,Piper,PA24-180,No,1,Reciprocating,Personal,4,0,0,0,Probable Cause
2,1974-08-30,Fatal(3),Destroyed,NaN,Cessna,172M,No,1,Reciprocating,Personal,3,NaN,NaN,NaN,Probable Cause
3,1977-06-19,Fatal(2),Destroyed,NaN,Rockwell,112,No,1,Reciprocating,Personal,2,0,0,0,Probable Cause
4,1979-08-02,Fatal(1),Destroyed,NaN,Cessna,501,No,NaN,NaN,Personal,1,2,NaN,0,Probable Cause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,2022-12-26,Minor,NaN,NaN,PIPER,PA-28-151,No,NaN,NaN,Personal,0,1,0,0,NaN
88885,2022-12-26,NaN,NaN,NaN,BELLANCA,7ECA,No,NaN,NaN,NaN,0,0,0,0,NaN
88886,2022-12-26,Non-Fatal,Substantial,Airplane,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1,NaN,Personal,0,0,0,1,NaN
88887,2022-12-26,NaN,NaN,NaN,CESSNA,210N,No,NaN,NaN,Personal,0,0,0,0,NaN
